<a href="https://colab.research.google.com/github/rirawax/data_science/blob/main/An%C3%A1lise_de_Pre%C3%A7os_de_medicamentos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Análise de dados de preços de medicamentos

Visando uma mudança próxima, resolvi fazer uma análise comparativa dos preços finais de medicamentos entre o estado que moro atualmente (Rio de Janeiro) e as outras opções que tenho em vista (São Paulo e Brasília)

Os dados foram extraídos do site do governo federal: 
https://dados.gov.br/dataset/preco-de-medicamentos-no-brasil-consumidor



In [2]:
!pip install statsmodels=='0.9.0'
import statsmodels as sm
sm.__version__
from statsmodels.stats.weightstats import ztest
from statsmodels.stats.weightstats import zconfint
from statsmodels.stats.weightstats import DescrStatsW
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import normaltest
from scipy.stats import ttest_ind
from scipy.stats import ranksums

     |████████████████████████████████| 7.4 MB 23.9 MB/s 
  Attempting uninstall: statsmodels
    Found existing installation: statsmodels 0.10.2
    Uninstalling statsmodels-0.10.2:
      Successfully uninstalled statsmodels-0.10.2


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


-Dataset para o primeiro teste de modelagem.

In [10]:
df = pd.read_csv("/content/drive/MyDrive/DataScience/TA_PRECO_MEDICAMENTO.csv", sep= ";", encoding="latin-1")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


-Dataset para o segundo teste de modelagem.

In [146]:
df2 = pd.read_csv("/content/drive/MyDrive/DataScience/TA_PRECO_MEDICAMENTO.csv", sep= ";", encoding="latin-1")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [12]:
df.head(10)

,SUBSTÂNCIA,CNPJ,LABORATÓRIO,CÓDIGO GGREM,REGISTRO,EAN 1,EAN 2,EAN 3,PRODUTO,APRESENTAÇÃO,...,PMC 18% ALC,PMC 20%,RESTRIÇÃO HOSPITALAR,CAP,CONFAZ 87,ICMS 0%,ANÁLISE RECURSAL,LISTA DE CONCESSÃO DE CRÉDITO TRIBUTÁRIO (PIS/COFINS),COMERCIALIZAÇÃO 2020,TARJA
0,SALICILATO DE FENILA;ÁCIDO SALICÍLICO;ÓXIDO DE...,33.379.884/0001-96,LABORATORIO SIMOES LTDA.,520500901178410,57600510011,7896210500354,-,-,TALCO ALÍVIO,TALQUEIRA C/ 100 G,...,"8,50","9,70",Não,Não,Não,Não,NaN,Negativa,Não,Tarja -(*)
1,DIENOGESTE;VALERATO DE ESTRADIOL,56.990.534/0001-67,SCHERING DO BRASIL QUÍMICA E FARMACÊUTICA LTDA,530917060013104,1002001390025,7896116871497,-,-,CHARMELIE,COM REV A + COM REV B + COM REV C + COM REV D ...,...,"163,25","186,12",Não,Não,Não,Não,NaN,Negativa,Não,Tarja Vermelha(*)
2,HIDROCLOROTIAZIDA;MALEATO DE ENALAPRIL,45.987.013/0001-34,ORGANON FARMACÊUTICA LTDA.,525500402113310,1002900020141,7897337705905,-,-,CO-RENITEC,"20 MG + 12,5 MG COM CT BL AL/AL X 30",...,"73,78","75,62",Não,Não,Não,Não,NaN,Positiva,Não,Tarja Vermelha
3,HIDROCLOROTIAZIDA;MALEATO DE ENALAPRIL,45.987.013/0001-34,ORGANON FARMACÊUTICA LTDA.,525500401117312,1002900020158,7897337705912,-,-,CO-RENITEC,10 MG + 25 MG COM CT BL AL/AL X 30,...,"48,45","49,67",Não,Não,Não,Não,NaN,Positiva,Não,Tarja Vermelha
4,MONTELUCASTE,45.987.013/0001-34,ORGANON FARMACÊUTICA LTDA.,525502006118219,1002900050014,7897337706575,-,-,SINGULAIR,5 MG COM MAST CT 3 BL AL PLAS INC X 10,...,"75,32","77,20",Não,Não,Não,Não,NaN,Positiva,Sim,Tarja Vermelha
5,MONTELUCASTE,45.987.013/0001-34,ORGANON FARMACÊUTICA LTDA.,525502002112216,1002900050022,7897337706568,-,-,SINGULAIR,10 MG COM REV CT 3 BL AL PLAS INC X 10,...,"75,32","77,20",Não,Não,Não,Não,NaN,Positiva,Sim,Tarja Vermelha
6,MONTELUCASTE,45.987.013/0001-34,ORGANON FARMACÊUTICA LTDA.,525502003119214,1002900050057,7897337706490,-,-,SINGULAIR,4 MG COM MAST CT BL AL PLAS INC X 10,...,"50,63","51,90",Não,Não,Não,Não,NaN,Positiva,Sim,Tarja Vermelha
7,MONTELUCASTE,45.987.013/0001-34,ORGANON FARMACÊUTICA LTDA.,525502004115212,1002900050065,7897337706506,-,-,SINGULAIR,4 MG COM MAST CT 3 BL AL PLAS INC X 10,...,"75,32","77,20",Não,Não,Não,Não,NaN,Positiva,Sim,Tarja Vermelha
8,MONTELUCASTE,45.987.013/0001-34,ORGANON FARMACÊUTICA LTDA.,525502005111210,1002900050073,7897337706520,-,-,SINGULAIR,5 MG COM MAST CT BL AL PLAS INC X 10,...,"50,63","51,90",Não,Não,Não,Não,NaN,Positiva,Sim,Tarja Vermelha
9,MONTELUCASTE,45.987.013/0001-34,ORGANON FARMACÊUTICA LTDA.,525502001116218,1002900050081,7897337706551,-,-,SINGULAIR,10 MG COM REV CT BL AL PLAS INC X 10,...,"50,63","51,90",Não,Não,Não,Não,NaN,Positiva,Sim,Tarja Vermelha


In [19]:
df.shape

(26310, 40)

In [21]:
df.dtypes

SUBSTÂNCIA                                               object
CNPJ                                                     object
LABORATÓRIO                                              object
CÓDIGO GGREM                                              int64
REGISTRO                                                 object
EAN 1                                                    object
EAN 2                                                    object
EAN 3                                                    object
PRODUTO                                                  object
APRESENTAÇÃO                                             object
CLASSE TERAPÊUTICA                                       object
TIPO DE PRODUTO (STATUS DO PRODUTO)                      object
REGIME DE PREÇO                                          object
PF Sem Impostos                                          object
PF 0%                                                    object
PF 12%                                  

In [116]:
tipos_de_tarja = df.TARJA
pmcRJ = df['PMC 20%']
pfRJ = df['PF 20%']

In [32]:
tipos_de_tarja.drop_duplicates (inplace=True)

In [33]:
tipos_de_tarja

0                           Tarja -(*)
1                    Tarja Vermelha(*)
2                      Tarja  Vermelha
48                  Tarja Vermelha (*)
229                  Tarja Venda Livre
238    Tarja Venda Livre/Sem Tarja (*)
265                       Tarja  Preta
459                             - (*) 
646                     Tarja Preta(*)
Name: TARJA, dtype: object

In [31]:
df['TIPO DE PRODUTO (STATUS DO PRODUTO)']

0             Similar
1             Similar
2                Novo
3                Novo
4                Novo
             ...     
26305        Genérico
26306      Biológicos
26307      Biológicos
26308    Fitoterápico
26309         Similar
Name: TIPO DE PRODUTO (STATUS DO PRODUTO), Length: 26310, dtype: object

In [36]:
restricao_hospitalar = df['RESTRIÇÃO HOSPITALAR']
restricao_hospitalar

0        Não
1        Não
2        Não
3        Não
4        Não
        ... 
26305    Sim
26306    Não
26307    Não
26308    Não
26309    Não
Name: RESTRIÇÃO HOSPITALAR, Length: 26310, dtype: object

In [61]:
df.rename(columns={"RESTRIÇÃO HOSPITALAR": "restricao"}, inplace=True)

In [88]:
index01 = df['SUBSTÂNCIA']
dados = df['restricao']
#dados.append(df['SUBSTÂNCIA'])
controle_substancia = pd.DataFrame (data = index01)

In [92]:
controle_substancia

,SUBSTÂNCIA,restricao
0,SALICILATO DE FENILA;ÁCIDO SALICÍLICO;ÓXIDO DE...,Não
1,DIENOGESTE;VALERATO DE ESTRADIOL,Não
2,HIDROCLOROTIAZIDA;MALEATO DE ENALAPRIL,Não
3,HIDROCLOROTIAZIDA;MALEATO DE ENALAPRIL,Não
4,MONTELUCASTE,Não
...,...,...
26305,CLORIDRATO DE SEVELÂMER,Sim
26306,INSULINA HUMANA,Não
26307,INSULINA HUMANA,Não
26308,ISOCONAZOL;ABARELIX,Não


In [91]:
controle_substancia = pd.concat([controle_substancia, dados], axis=1)

In [112]:
subst_hospitalar = controle_substancia.query('restricao == "Sim"')

In [113]:
subst_hospitalar.drop_duplicates(subset=['SUBSTÂNCIA'], inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [114]:
subst_hospitalar.reset_index(drop=True, inplace=True)

In [115]:
subst_hospitalar

,SUBSTÂNCIA,restricao
0,ACETATO DE CASPOFUNGINA,Sim
1,ERTAPENÉM SÓDICO,Sim
2,FOSAPREPITANTO DIMEGLUMINA,Sim
3,PEMBROLIZUMABE,Sim
4,DAPTOMICINA,Sim
...,...,...
751,PALIVIZUMABE,Sim
752,BERACTANTO,Sim
753,CALCITRIOL,Sim
754,CABERGOLINA,Sim


No primeiro teste fiz uma seleção de quais eram os medicamentos de uso hospitalar, mas achei que poderia tomar outro caminho e fazer uma limpeza mais simples.

In [147]:
df2.drop(columns=['CNPJ','CLASSE TERAPÊUTICA','PF 17% ALC','PMC 17% ALC','PF 17,5% ALC','PMC 17,5% ALC','PF 18% ALC','PMC 18% ALC','CÓDIGO GGREM','EAN 1','EAN 2','EAN 3','CAP','ANÁLISE RECURSAL','LISTA DE CONCESSÃO DE CRÉDITO TRIBUTÁRIO (PIS/COFINS)'], inplace=True)

In [148]:
df2.dtypes

SUBSTÂNCIA                             object
LABORATÓRIO                            object
REGISTRO                               object
PRODUTO                                object
APRESENTAÇÃO                           object
TIPO DE PRODUTO (STATUS DO PRODUTO)    object
REGIME DE PREÇO                        object
PF Sem Impostos                        object
PF 0%                                  object
PF 12%                                 object
PF 17%                                 object
PF 17,5%                               object
PF 18%                                 object
PF 20%                                 object
PMC 0%                                 object
PMC 12%                                object
PMC 17%                                object
PMC 17,5%                              object
PMC 18%                                object
PMC 20%                                object
RESTRIÇÃO HOSPITALAR                   object
CONFAZ 87                         

In [151]:
df2.shape

(26310, 25)

In [150]:
df2.drop_duplicates()

,SUBSTÂNCIA,LABORATÓRIO,REGISTRO,PRODUTO,APRESENTAÇÃO,TIPO DE PRODUTO (STATUS DO PRODUTO),REGIME DE PREÇO,PF Sem Impostos,PF 0%,PF 12%,...,PMC 12%,PMC 17%,"PMC 17,5%",PMC 18%,PMC 20%,RESTRIÇÃO HOSPITALAR,CONFAZ 87,ICMS 0%,COMERCIALIZAÇÃO 2020,TARJA
0,SALICILATO DE FENILA;ÁCIDO SALICÍLICO;ÓXIDO DE...,LABORATORIO SIMOES LTDA.,57600510011,TALCO ALÍVIO,TALQUEIRA C/ 100 G,Similar,Regulado,"5,04","5,65","6,53",...,"8,72","9,30","9,37","9,43","9,70",Não,Não,Não,Não,Tarja -(*)
1,DIENOGESTE;VALERATO DE ESTRADIOL,SCHERING DO BRASIL QUÍMICA E FARMACÊUTICA LTDA,1002001390025,CHARMELIE,COM REV A + COM REV B + COM REV C + COM REV D ...,Similar,Regulado,"96,83","108,45","125,32",...,"167,40","178,63","179,84","181,06","186,12",Não,Não,Não,Não,Tarja Vermelha(*)
2,HIDROCLOROTIAZIDA;MALEATO DE ENALAPRIL,ORGANON FARMACÊUTICA LTDA.,1002900020141,CO-RENITEC,"20 MG + 12,5 MG COM CT BL AL/AL X 30",Novo,Regulado,"43,76","43,76","49,73",...,"68,75","72,88","73,32","73,78","75,62",Não,Não,Não,Não,Tarja Vermelha
3,HIDROCLOROTIAZIDA;MALEATO DE ENALAPRIL,ORGANON FARMACÊUTICA LTDA.,1002900020158,CO-RENITEC,10 MG + 25 MG COM CT BL AL/AL X 30,Novo,Regulado,"28,74","28,74","32,66",...,"45,15","47,87","48,16","48,45","49,67",Não,Não,Não,Não,Tarja Vermelha
4,MONTELUCASTE,ORGANON FARMACÊUTICA LTDA.,1002900050014,SINGULAIR,5 MG COM MAST CT 3 BL AL PLAS INC X 10,Novo,Regulado,"44,67","44,67","50,76",...,"70,17","74,40","74,86","75,32","77,20",Não,Não,Não,Sim,Tarja Vermelha
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26305,CLORIDRATO DE SEVELÂMER,FUNDAÇÃO BAIANA DE PESQ. CIENTIFICA E DESENV. ...,1988300020011,CLORIDRATO DE SEVELAMER,800 MG COM REV CT FR PLAS OPC X 180,Genérico,Regulado,"503,12","563,50",NaN,...,NaN,NaN,NaN,NaN,NaN,Sim,Sim,Sim,Não,Tarja Vermelha(*)
26306,INSULINA HUMANA,FUNDAÇÃO BAIANA DE PESQ. CIENTIFICA E DESENV. ...,1988300040012,BAHIAFARMA INSULINA HUMANA NPH,100 UI/ML SUS INJ CT FA VD INC X 10 ML,Biológicos,Regulado,"32,80","32,80",NaN,...,NaN,NaN,NaN,NaN,NaN,Não,Sim,Sim,Sim,Tarja Vermelha(*)
26307,INSULINA HUMANA,FUNDAÇÃO BAIANA DE PESQ. CIENTIFICA E DESENV. ...,1988300050018,BAHIAFARMA INSULINA HUMANA R,100 UI/ML SOL INJ CT FA VD INC X 10 ML,Biológicos,Regulado,"34,22","34,22",NaN,...,NaN,NaN,NaN,NaN,NaN,Não,Sim,Sim,Sim,Tarja Vermelha
26308,ISOCONAZOL;ABARELIX,VISA ESTADUAL SP TESTE,9965400350064,PRODUTO DE MEDICAMENTO PARA TESTE,APRESENTAÇÃO 01&nbsp;03,Fitoterápico,Regulado,"7,32","8,20","9,48",...,"12,66","13,50","13,59","13,70","14,07",Não,Sim,Não,Não,Tarja Vermelha


In [160]:
df2.dropna()

,SUBSTÂNCIA,LABORATÓRIO,REGISTRO,PRODUTO,APRESENTAÇÃO,TIPO DE PRODUTO (STATUS DO PRODUTO),REGIME DE PREÇO,PF Sem Impostos,PF 0%,PF 12%,...,PMC 12%,PMC 17%,"PMC 17,5%",PMC 18%,PMC 20%,RESTRIÇÃO HOSPITALAR,CONFAZ 87,ICMS 0%,COMERCIALIZAÇÃO 2020,TARJA
0,SALICILATO DE FENILA;ÁCIDO SALICÍLICO;ÓXIDO DE...,LABORATORIO SIMOES LTDA.,57600510011,TALCO ALÍVIO,TALQUEIRA C/ 100 G,Similar,Regulado,"5,04","5,65","6,53",...,"8,72","9,30","9,37","9,43","9,70",Não,Não,Não,Não,Tarja -(*)
1,DIENOGESTE;VALERATO DE ESTRADIOL,SCHERING DO BRASIL QUÍMICA E FARMACÊUTICA LTDA,1002001390025,CHARMELIE,COM REV A + COM REV B + COM REV C + COM REV D ...,Similar,Regulado,"96,83","108,45","125,32",...,"167,40","178,63","179,84","181,06","186,12",Não,Não,Não,Não,Tarja Vermelha(*)
2,HIDROCLOROTIAZIDA;MALEATO DE ENALAPRIL,ORGANON FARMACÊUTICA LTDA.,1002900020141,CO-RENITEC,"20 MG + 12,5 MG COM CT BL AL/AL X 30",Novo,Regulado,"43,76","43,76","49,73",...,"68,75","72,88","73,32","73,78","75,62",Não,Não,Não,Não,Tarja Vermelha
3,HIDROCLOROTIAZIDA;MALEATO DE ENALAPRIL,ORGANON FARMACÊUTICA LTDA.,1002900020158,CO-RENITEC,10 MG + 25 MG COM CT BL AL/AL X 30,Novo,Regulado,"28,74","28,74","32,66",...,"45,15","47,87","48,16","48,45","49,67",Não,Não,Não,Não,Tarja Vermelha
4,MONTELUCASTE,ORGANON FARMACÊUTICA LTDA.,1002900050014,SINGULAIR,5 MG COM MAST CT 3 BL AL PLAS INC X 10,Novo,Regulado,"44,67","44,67","50,76",...,"70,17","74,40","74,86","75,32","77,20",Não,Não,Não,Sim,Tarja Vermelha
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26300,RISANQUIZUMABE,ABBVIE FARMACÊUTICA LTDA.,1986000160016,SKYRIZI,75 MG SOL INJ CT 2 BL X SER PREENC VD TRANS X ...,Biológico,Regulado,"13215,15","14800,97","17103,82",...,"22847,01","24379,15","24543,80","24710,67","25401,76",Não,Não,Não,Sim,Tarja Vermelha(*)
26301,UPADACITINIBE HEMI-HIDRATADO,ABBVIE FARMACÊUTICA LTDA.,1986000170011,RINVOQ,15 MG COM REV LIB PROL CT FR PLAS PEAD OPC X 30,Novo,Regulado,"3489,06","3907,75","4515,75",...,"6032,07","6436,59","6480,05","6524,10","6706,57",Não,Não,Não,Sim,Tarja Vermelha(*)
26302,UPADACITINIBE HEMI-HIDRATADO,ABBVIE FARMACÊUTICA LTDA.,1986000170021,RINVOQ,15 MG COM REV LIB PROL CT 3 FR PLAS PEAD OPC X 30,Novo,Regulado,"10467,18","11723,24","13547,24",...,"18096,19","19309,73","19440,14","19572,30","20119,69",Não,Não,Não,Não,Tarja Vermelha(*)
26308,ISOCONAZOL;ABARELIX,VISA ESTADUAL SP TESTE,9965400350064,PRODUTO DE MEDICAMENTO PARA TESTE,APRESENTAÇÃO 01&nbsp;03,Fitoterápico,Regulado,"7,32","8,20","9,48",...,"12,66","13,50","13,59","13,70","14,07",Não,Sim,Não,Não,Tarja Vermelha


In [166]:
df2.mean()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  """Entry point for launching an IPython kernel.


Series([], dtype: float64)